In [1]:
import pandas as pd
df = pd.read_csv('archive/Korea Income and Welfare.csv')
df.head()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
0,10101,2005,1,1,614.0,1,2,1936,2,2,2,,,8
1,10101,2011,7,1,896.0,1,2,1936,2,2,2,,,10
2,10101,2012,8,1,1310.0,1,2,1936,2,2,2,,,10
3,10101,2013,9,1,2208.0,1,2,1936,2,2,2,,,1
4,10101,2014,10,1,864.0,1,2,1936,2,2,2,,,10


In [2]:
df.describe()

,id,year,wave,region,income,family_member,gender,year_born,education_level,marriage,religion
count,9.285700e+04,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000,92857.000000
mean,4.365181e+07,2011.517958,7.517958,3.691353,3441.122327,2.484304,1.296316,1952.956740,4.511895,1.679389,1.499521
std,2.619007e+07,4.011222,4.011222,2.058961,4176.324522,1.292251,0.456634,16.006065,1.669887,1.103413,0.531572
min,1.010100e+04,2005.000000,1.000000,1.000000,-232174.000000,1.000000,1.000000,1910.000000,2.000000,0.000000,1.000000
25%,2.177120e+07,2008.000000,4.000000,2.000000,1140.000000,1.000000,1.000000,1939.000000,3.000000,1.000000,1.000000
50%,4.322010e+07,2012.000000,8.000000,3.000000,2428.000000,2.000000,1.000000,1952.000000,5.000000,1.000000,1.000000
75%,6.190030e+07,2015.000000,11.000000,5.000000,4695.000000,3.000000,2.000000,1966.000000,5.000000,2.000000,2.000000
max,9.800070e+07,2018.000000,14.000000,7.000000,468209.000000,9.000000,2.000000,2002.000000,9.000000,9.000000,9.000000


In [3]:
df.isnull().sum()

id                    0
year                  0
wave                  0
region                0
income                0
family_member         0
gender                0
year_born             0
education_level       0
marriage              0
religion              0
occupation            0
company_size          0
reason_none_worker    0
dtype: int64

In [4]:
avg_income_region = df.groupby('region')['income'].mean()
print(avg_income_region)

region
1    4028.787273
2    3907.235626
3    3433.993306
4    2771.124671
5    3742.423781
6    3081.881088
7    2870.977443
Name: income, dtype: float64


In [6]:
family_member_education = df.groupby('education_level')['family_member'].sum()
print(family_member_education)

education_level
2    15546
3    40277
4    28743
5    77809
6    18074
7    42382
8     7094
9      760
Name: family_member, dtype: int64


In [7]:
gender_religion = df.groupby(['gender', 'religion'])['marriage'].value_counts(normalize=True)
print(gender_religion)

gender  religion  marriage
1       1         1           0.887081
                  5           0.037929
                  3           0.036693
                  2           0.032549
                  4           0.005080
                  6           0.000568
                  0           0.000067
                  9           0.000033
        2         1           0.814362
                  5           0.073169
                  3           0.062030
                  2           0.040260
                  4           0.009217
                  6           0.000679
                  0           0.000226
                  9           0.000057
        9         1           0.875000
                  2           0.041667
                  5           0.041667
                  0           0.020833
                  6           0.020833
2       1         2           0.712944
                  3           0.151356
                  1           0.062563
                  5           0.04743

In [25]:
q3 = df.quantile(0.75, numeric_only=True)
q1 = df.quantile(0.25, numeric_only=True)

iqr = q3 - q1

def is_outlier(df):
    score = df['income']
    if score > q3['income'] + 1.5*iqr['income'] or score < q1['income'] - 1.5 * iqr['income']:
        return True
    else:
        return False

df['income_outlier'] = df.apply(is_outlier, axis = 1)

df = df.loc[df['income_outlier'] == False]

del df['income_outlier']

df['income'].value_counts()

3000.0    71
2400.0    70
1800.0    68
1200.0    57
840.0     56
          ..
7499.0     1
7105.0     1
7987.0     1
2544.4     1
1446.3     1
Name: income, Length: 9356, dtype: int64

In [26]:
df_job_on = df[(df['occupation'] != ' ') & (df['reason_none_worker'] == ' ') ]

df_job_on = df_job_on[(df_job_on['occupation'] != ' ') & (df_job_on['company_size'] != ' ') ]

df_job_off = df[(df['occupation'] == ' ') & (df['reason_none_worker'] != ' ') ]

df_job_on.head()

,wave,region,income,family_member,gender,year_born,education_level,marriage,religion,occupation,company_size,reason_none_worker
8,1,1,1257.0,1,2,1945,4,2,2,421,1,
10,3,1,602.0,1,2,1945,4,2,2,411,1,
11,4,1,1972.0,1,2,1945,4,2,2,951,1,
12,5,1,1638.0,1,2,1945,4,2,1,951,1,
13,6,1,1598.0,1,2,1945,4,2,2,951,1,


In [28]:
df_soldier = df_job_on[(df_job_on['occupation'] == '1011') | (df_job_on['occupation'] == '1012')]
df_soldier['occupation'] = '100'

soldier_num = df_job_on[(df_job_on['occupation'] == '1011') | (df_job_on['occupation'] == '1012')].index
df_job_on.drop(soldier_num, inplace = True)

In [29]:
df_job_on['occupation'] = df_job_on['occupation'].str[0] * 2

df_job_on = pd.concat([df_job_on,df_soldier])
df_job_on['occupation'].value_counts()

66    11106
99    10770
88     6719
77     5753
33     5187
22     4517
55     4217
44     3700
11     1893
Name: occupation, dtype: int64

In [30]:
df_job_off['reason_none_worker'].replace('11','0', inplace=True)

num = df_job_off[df_job_off['reason_none_worker'] == '99'].index

df_job_off.drop(num, inplace=True)
df_job_off['reason_none_worker'].value_counts()

C:\Users\wjsdl\AppData\Local\Temp\ipykernel_18908\3490302467.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_off['reason_none_worker'].replace('11','0', inplace=True)
C:\Users\wjsdl\AppData\Local\Temp\ipykernel_18908\3490302467.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_job_off.drop(num, inplace=True)


10    18934
8      4234
9      2657
1      2480
7      1114
6      1059
4       701
5       322
0       231
3        49
2         7
Name: reason_none_worker, dtype: int64

In [31]:
df.drop(['id','year'], axis = 1, inplace = True)

df = df[df['income'] >= 0]

KeyError: "['id', 'year'] not found in axis"

In [35]:
num1 = df[(df['marriage'] == 0) | (df['marriage'] == 9)].index
df.drop(num1, inplace =True)

num2 = df[df['religion'] == 9].index
df.drop(num2, inplace=True)

num3 = df[df['company_size']=='99'].index
df.drop(num3, inplace=True)

In [36]:
print(df['region'].value_counts())
print(df['gender'].value_counts())
print(df['education_level'].value_counts())
print(df['marriage'].value_counts())
print(df['religion'].value_counts())

2    18062
7    15233
3    15093
1    13195
4    11741
5     7237
6     6626
Name: region, dtype: int64
1    59886
2    27301
Name: gender, dtype: int64
5    24378
3    20978
4    11982
7    11719
2    10826
6     5420
8     1705
9      179
Name: education_level, dtype: int64
1    51825
2    21130
3     7351
5     5685
4     1142
6       54
Name: marriage, dtype: int64
1    44068
2    43119
Name: religion, dtype: int64


In [37]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87187 entries, 0 to 92856
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   wave                87187 non-null  int64  
 1   region              87187 non-null  int64  
 2   income              87187 non-null  float64
 3   family_member       87187 non-null  int64  
 4   gender              87187 non-null  int64  
 5   year_born           87187 non-null  int64  
 6   education_level     87187 non-null  int64  
 7   marriage            87187 non-null  int64  
 8   religion            87187 non-null  int64  
 9   occupation          87187 non-null  object 
 10  company_size        87187 non-null  object 
 11  reason_none_worker  87187 non-null  object 
dtypes: float64(1), int64(8), object(3)
memory usage: 8.6+ MB


In [41]:
df['region'] = df['region'].apply(str)

df['company_size'] = pd.to_numeric(df['company_size'], errors='coerce').fillna(0)

df['gender'].replace([1, 2], [0, 1], inplace=True)
df['religion'].replace([1, 2], [0, 1], inplace=True)

df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87187 entries, 0 to 92856
Data columns (total 12 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   wave                87187 non-null  int64  
 1   region              87187 non-null  object 
 2   income              87187 non-null  float64
 3   family_member       87187 non-null  int64  
 4   gender              87187 non-null  int64  
 5   year_born           87187 non-null  int64  
 6   education_level     87187 non-null  int64  
 7   marriage            87187 non-null  int64  
 8   religion            87187 non-null  int64  
 9   occupation          87187 non-null  object 
 10  company_size        87187 non-null  float64
 11  reason_none_worker  87187 non-null  object 
dtypes: float64(2), int64(7), object(3)
memory usage: 8.6+ MB


In [42]:
cal_cols = df.select_dtypes('object').columns.values

df = pd.get_dummies(data=df, columns=cal_cols)
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 87187 entries, 0 to 92856
Columns: 273 entries, wave to reason_none_worker_99
dtypes: float64(2), int64(7), uint8(264)
memory usage: 28.6 MB


In [43]:
df.scribe()

AttributeError: 'DataFrame' object has no attribute 'scribe'

In [44]:
df.head()

,wave,income,family_member,gender,year_born,education_level,marriage,religion,company_size,region_1,...,reason_none_worker_11,reason_none_worker_2,reason_none_worker_3,reason_none_worker_4,reason_none_worker_5,reason_none_worker_6,reason_none_worker_7,reason_none_worker_8,reason_none_worker_9,reason_none_worker_99
0,1,614.0,1,1,1936,2,2,1,0.0,1,...,0,0,0,0,0,0,0,1,0,0
1,7,896.0,1,1,1936,2,2,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
2,8,1310.0,1,1,1936,2,2,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
3,9,2208.0,1,1,1936,2,2,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0
4,10,864.0,1,1,1936,2,2,1,0.0,1,...,0,0,0,0,0,0,0,0,0,0


In [45]:
x = df.drop('income', axis = 1).values
y = df['income'].values

x.shape, y.shape

((87187, 272), (87187,))

In [46]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.3, random_state=42)

In [ ]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()

x_train = scaler.fit_transform(x_train)
x_test = scaler =